In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
trainImages = open(os.getcwd() + '/data/MINST/train-images.idx3-ubyte', 'rb')
trainLabels = open(os.getcwd() + '/data/MINST/train-labels.idx1-ubyte', 'rb')
testImages = open(os.getcwd() + '/data/MINST/t10k-images.idx3-ubyte', 'rb')
testLabels = open(os.getcwd() + '/data/MINST/t10k-labels.idx1-ubyte', 'rb')

In [5]:
assert(int.from_bytes(trainImages.read(4), "big") == 2051)
assert(int.from_bytes(trainLabels.read(4), 'big') == 2049)
assert(int.from_bytes(testImages.read(4), 'big') == 2051)
assert(int.from_bytes(testLabels.read(4), 'big') == 2049)

In [6]:
trainImagesNum = int.from_bytes(trainImages.read(4), 'big')
trainLabelsNum = int.from_bytes(trainLabels.read(4), 'big')
testLabelsNum = int.from_bytes(testLabels.read(4), 'big')
testImagesNum = int.from_bytes(testImages.read(4), 'big')

In [7]:
assert(trainImagesNum == trainLabelsNum)
assert(testImagesNum == testLabelsNum)

In [8]:
trainRows = int.from_bytes(trainImages.read(4), 'big')
trainCols = int.from_bytes(trainImages.read(4), 'big')
testRows = int.from_bytes(testImages.read(4), 'big')
testCols = int.from_bytes(testImages.read(4), 'big')

In [9]:
trainPixels = trainCols * trainRows
testPixels = testCols * testRows

In [10]:
trainX = np.zeros((trainImagesNum, trainPixels))
trainY = np.zeros(trainLabelsNum)
testX = np.zeros((testImagesNum, testPixels))
testY = np.zeros((testLabelsNum))

In [11]:
for i in range(trainImagesNum):
    for j in range(trainPixels):
        trainX[i,j] = float(int.from_bytes(trainImages.read(1), 'big'))
    trainY[i] = int.from_bytes(trainLabels.read(1), 'big')

In [12]:
for i in range(testImagesNum):
    for j in range(testPixels):
        testX[i,j] = float(int.from_bytes(testImages.read(1), 'big'))
    testY[i] = int.from_bytes(testLabels.read(1), 'big')

In [13]:
trainX = trainX/255
testX = testX/255

In [14]:
trainX = tf.convert_to_tensor(trainX)
trainY = tf.keras.utils.to_categorical(trainY, num_classes=10)
testX = tf.convert_to_tensor(testX)
testY = tf.keras.utils.to_categorical(testY, num_classes=10)

In [15]:
def simpleModel(input_shape):
    XInput = tf.keras.Input(shape=input_shape)
    X = tf.keras.layers.Dense(200, activation='relu', name='fc0')(XInput)
    X = tf.keras.layers.Dropout(0.2)(X)
    X = tf.keras.layers.Dense(200, activation='relu', name='fc1')(X)
    X = tf.keras.layers.Dropout(0.2)(X)
    X = tf.keras.layers.Dense(10,activation='softmax', name ='out')(X)
    model = tf.keras.Model(inputs=XInput, outputs = X, name = 'SimpleModel')
    return model

In [16]:
model = simpleModel(trainX.shape[1:])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

In [17]:
preds = model.fit(x=trainX, y=trainY, batch_size= 100, epochs=50, verbose=1, sample_weight=None)

Epoch 1/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0531 - accuracy: 0.9042
Epoch 2/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0244 - accuracy: 0.9578
Epoch 3/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0185 - accuracy: 0.9686
Epoch 4/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0149 - accuracy: 0.9743
Epoch 5/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0128 - accuracy: 0.9779
Epoch 6/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0109 - accuracy: 0.9807
Epoch 7/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0098 - accuracy: 0.9834
Epoch 8/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0090 - accuracy: 0.9852
Epoch 9/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0082 - accuracy: 0.9859
Epoch 10/50
600/600 [==============================] - 1s 2ms/step - loss: 0.0076 - accuracy: 0.9865

In [18]:
preds = model.evaluate(testX, testY, batch_size=100, verbose=1, sample_weight=None)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

100/100 [==============================] - 0s 1ms/step - loss: 0.0154 - accuracy: 0.9832

Loss = 0.015430285595357418
Test Accuracy = 0.9832000136375427


In [19]:
def convModel(input_shape):
    XIn = tf.keras.Input(shape=input_shape)
    X = tf.keras.layers.Convolution2D(5, 3, data_format='channels_last',input_shape=input_shape, padding='same', activation='relu')(XIn)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)(X)
    Y = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Convolution2D(10, 3, padding='valid', activation='relu')(Y)
    X = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)(X)
    X = tf.keras.layers.BatchNormalization()(X)
    Y = tf.keras.layers.Convolution2D(10, 2)(Y)
    Y = tf.keras.layers.MaxPool2D(pool_size=(2,2))(Y)
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dropout(0.2)(X)
    X = tf.keras.layers.Dense(100, activation='relu')(X)
    X = tf.keras.layers.Dense(10, activation='softmax')(X)
    model = tf.keras.Model(inputs=XIn, outputs=X)
    return model

In [20]:
trainX = tf.reshape(trainX, shape=(trainX.shape[0], 28,28,1))
testX = tf.reshape(testX, shape=(testX.shape[0], 28,28,1))

In [21]:
model2 = convModel(trainX.shape[1:])
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [22]:
model2.fit(x=trainX, y=trainY, batch_size= 100, epochs=50, verbose=1, sample_weight=None)

Epoch 1/50
600/600 [==============================] - 2s 3ms/step - loss: 0.2426 - accuracy: 0.9237
Epoch 2/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0867 - accuracy: 0.9728
Epoch 3/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0620 - accuracy: 0.9805
Epoch 4/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0516 - accuracy: 0.9831
Epoch 5/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0436 - accuracy: 0.9858
Epoch 6/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0392 - accuracy: 0.9873
Epoch 7/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0352 - accuracy: 0.9882
Epoch 8/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0309 - accuracy: 0.9900
Epoch 9/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0288 - accuracy: 0.9906
Epoch 10/50
600/600 [==============================] - 2s 3ms/step - loss: 0.0252 - accuracy: 0.9914

In [32]:
predictions = model2.evaluate(testX, testY, batch_size=100, verbose=1, sample_weight=None)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

100/100 [==============================] - 0s 2ms/step - loss: 0.0418 - accuracy: 0.9904

Loss = 0.04175882413983345
Test Accuracy = 0.9904000163078308


In [33]:
predictions = model2.predict(testX)

In [34]:
print(predictions)

[[1.5188805e-13 2.9186197e-13 1.2718685e-13 ... 1.0000000e+00
  3.8355916e-12 8.2651969e-10]
 [2.7261662e-13 3.1574078e-08 1.0000000e+00 ... 7.1854518e-13
  5.5631848e-16 2.3113854e-18]
 [2.1894200e-10 1.0000000e+00 9.1491615e-13 ... 9.0871790e-09
  2.0683053e-09 5.0506257e-12]
 ...
 [5.1060467e-28 5.3437749e-18 2.9924255e-23 ... 8.8097703e-15
  1.5792810e-16 6.4195428e-14]
 [7.8469202e-12 1.3211989e-19 2.9035569e-16 ... 1.4393017e-16
  2.4898052e-08 1.1262410e-14]
 [4.6202482e-12 3.1473329e-14 3.6151362e-15 ... 2.0072403e-25
  4.8433252e-13 2.3891729e-18]]


In [38]:
pred = tf.argmax(predictions, axis=1)
pred

<tf.Tensor: shape=(10000,), dtype=int64, numpy=array([7, 2, 1, ..., 4, 5, 6])>

In [39]:
labels = tf.argmax(testY, axis=1)
labels

<tf.Tensor: shape=(10000,), dtype=int64, numpy=array([7, 2, 1, ..., 4, 5, 6])>

In [40]:
confusionMatrix = tf.math.confusion_matrix(labels,pred,10)
print(confusionMatrix)

tf.Tensor(
[[ 974    1    1    0    0    0    2    2    0    0]
 [   0 1133    0    0    0    0    0    1    1    0]
 [   1    2 1025    0    0    0    0    3    1    0]
 [   0    0    0 1007    0    3    0    0    0    0]
 [   0    0    1    0  972    0    0    2    0    7]
 [   1    0    0    2    0  887    2    0    0    0]
 [   4    2    1    0    4    2  944    0    0    1]
 [   0    3    9    0    0    0    0 1012    1    3]
 [   1    0    3    2    0    2    0    1  961    4]
 [   0    1    1    1    8    1    0    7    1  989]], shape=(10, 10), dtype=int32)


In [41]:
from sklearn.metrics import f1_score, recall_score, precision_score
f1 = f1_score(labels, pred, average='macro')
recall = recall_score(labels, pred, average='macro')
precision = precision_score(labels, pred, average='macro')
f1, recall, precision

(0.9903860850803005, 0.9903226903984319, 0.9904670367397521)